# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2023-spring@googlegroups.com

Slide:    [Link](https://docs.google.com/presentation/d/18LkR8qulwSbi3SVoLl1XNNGjQQ_qczs_35lrJWOmHCk/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/t/c76950cc460140eba30a576ca7668d28)

# Set up the environment


## Package installation

In [19]:
# Training progress bar
!pip install -q qqdm

## Downloading data

In [20]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh |  bash
!apt-get install -y --allow-unauthenticated git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.3.0).
0 upgraded, 0 newly installed, 0 to remove and 88 not upgraded.


In [21]:
!git clone https://github.com/chiyuanhsiao/ml2023spring-hw8

Cloning into 'ml2023spring-hw8'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 2), reused 8 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), done.
Resolving deltas: 100% (2/2), done.
Filtering content: 100% (2/2), 1.36 GiB | 30.79 MiB/s, done.


In [22]:
%cd /kaggle/working/ml2023spring-hw8
!git lfs install
!git lfs pull

/kaggle/working/ml2023spring-hw8
Updated Git hooks.
Git LFS initialized.


# Import packages

In [23]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [24]:
train = np.load('/kaggle/working/ml2023spring-hw8/trainingset.npy', allow_pickle=True)[:,8:56,8:56,:]
test = np.load('/kaggle/working/ml2023spring-hw8/testingset.npy', allow_pickle=True)[:,8:56,8:56,:]

print(train.shape)
print(test.shape)

(100000, 48, 48, 3)
(19636, 48, 48, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [25]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(44)

# Autoencoder

# Models & loss

In [26]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(48 * 48 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),  
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 10)
        
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(10, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 48 * 48 * 3),
            nn.Tanh()
        )
 
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# maybe it can be smaller
class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),        
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Flatten(1,3),
            nn.Linear(48*6*6, 48),
            nn.ReLU(),
        )
#         self.fce = nn.Sequential(
            
#         )
#         self.fcd = nn.Sequential(
            
#         )
        self.decoder = nn.Sequential(
            nn.Linear(48, 48*6*6),
            nn.ReLU(),
            nn.Unflatten(1, (48,6,6)),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
#         x = self.fce(x)
#         x = self.fcd(x)
        x = self.decoder(x)
        return x




class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),    
            nn.ReLU(),
        )
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
			      nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),
			      nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), 
            nn.Tanh(),
        )

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [27]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [28]:
class Residual_Block(nn.Module):
    def __init__(self, ic, oc, stride=1):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(ic, oc, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(oc),
            nn.ReLU(inplace=True)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(oc, oc, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(oc),
        )
        
        self.relu = nn.ReLU(inplace=True)
    
        self.downsample = None
        if stride != 1 or (ic != oc):
            self.downsample = nn.Sequential(
                nn.Conv2d(ic, oc, kernel_size=1, stride=stride),
                nn.BatchNorm2d(oc),
            )
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample:
            residual = self.downsample(x)
            
        out += residual
        return self.relu(out)
    
class ResNet(nn.Module):
    def __init__(self, block=Residual_Block, num_layers=[2, 1, 1, 1]):
        super().__init__()
        self.preconv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.layer0 = self.make_residual(block, 32, 64,  num_layers[0], stride=2)
        self.layer1 = self.make_residual(block, 64, 128, num_layers[1], stride=2)
        self.layer2 = self.make_residual(block, 128, 128, num_layers[2], stride=2)
        self.layer3 = self.make_residual(block, 128, 64, num_layers[3], stride=2)
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(36*4*4, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(64, 64*4*4),
            nn.BatchNorm1d(64*4*4),
            nn.ReLU(),
            nn.Unflatten(1, (64, 4, 4)),
            nn.ConvTranspose2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
    def make_residual(self, block, ic, oc, num_layer, stride=1):
        layers = []
        layers.append(block(ic, oc, stride))
        for i in range(1, num_layer):
            layers.append(block(oc, oc))
        return nn.Sequential(*layers)
    
    def encoder(self, x):
        x = self.preconv(x)
        x = self.layer0(x) #64*64 --> 32*32
        x = self.layer1(x) #32*32 --> 16*16
        x = self.layer2(x) #16*16 --> 8*8
        x = self.layer3(x) #8*8 --> 4*4
        x = self.fc(x)
        return x
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
# 完全就是把ResNet的decoder部分拿了过来
class Auxiliary(nn.Module):
    def __init__(self):
        super().__init__()
#         self.fcd = nn.Sequential(
            
#         )
        self.decoder = nn.Sequential(
            nn.Linear(48, 48*6*6),
            nn.ReLU(),
            nn.Unflatten(1, (48,6,6)),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        
    def forward(self, x):
#         x = self.fcd(x)
        return self.decoder(x)



In [29]:
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler
# Training hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 8e-4

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model
model_type = 'fcn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_type2 = 'fcn'
model_type3 = 'cnn'
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE(), 'resnet' : ResNet() }
model = model_classes[model_type].to('cuda')
model2 = model_classes[model_type2].to('cuda')
model3 = model_classes[model_type3].to('cuda')
models = [model, model2, model3]

aux = Auxiliary().to('cuda')

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay = 5e-4)
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=learning_rate, weight_decay = 1e-3)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=learning_rate)
optimizer_a = torch.optim.AdamW(aux.parameters(), lr=learning_rate, weight_decay = 5e-4)
optimizers = [optimizer, optimizer2, optimizer3]
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 4, T_mult = 2, eta_min = learning_rate/20)
scheduler_a = lr_scheduler.CosineAnnealingWarmRestarts(optimizer_a, T_0 = 2, T_mult = 2,eta_min = learning_rate/20)

## Training loop

In [18]:
best_loss = np.inf
# model.to('cuda')
model.train()

aux.train()
qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    tot_loss_a = list()
    for data in train_dataloader:
        temperature = epoch // 2 + 1
        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)
        

        # ===================forward=====================
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)
        
        tot_loss.append(loss.item())
        # ===================backward====================
#         torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #====================new=========================
#         model.eval()
#         z = model.encoder(img).detach_()
    
#         output = output.detach_()
#         output_a = aux(z)
#         loss_a = (criterion(output_a, output).mul(temperature).exp())*criterion(output_a, img)
#         tot_loss_a.append(loss_a)
#         optimizer_a.zero_grad()
#         loss_a.backward()
#         optimizer_a.step()
    scheduler.step()
#     scheduler_a.step()
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
#         torch.save(aux, 'best_model_{}.pt'.format('aux'))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))
#     torch.save(aux, 'la÷st_model_{}.pt'.format('aux'))

 Iters    Elapsed Time      Speed                                               
 0/100          -             -                                                 
Description   0.0% |                                                           |/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64, 3, 48, 48])) that is different to the input size (torch.Size([64, 3, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 Iters    Elapsed Time      Speed                                               
 0/100          -             -                                                 
Description   0.0% |                                                           |

RuntimeError: The size of tensor a (64) must match the size of tensor b (48) at non-singleton dimension 3

In [ ]:
# best_loss = np.inf
# model2.train()
# # aux.train()
# num_epochs = 240
# qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
# learning_rate = 8e-4
# scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer2, T_0 = 3, T_mult = 2)
# for epoch in qqdm_train:
#     tot_loss = list()
#     tot_loss_a = list()
#     for data in train_dataloader:
#         temperature = epoch // 2 + 1
#         # ===================loading=====================
#         img = data.float().cuda()
#         if model_type2 in ['fcn']:
#             img = img.view(img.shape[0], -1)
        

#         # ===================forward=====================
#         output = model2(img)
#         if model_type2 in ['vae']:
#             loss = loss_vae(output[0], img, output[1], output[2], criterion)
#         else:
#             loss = criterion(output, img)

#         tot_loss.append(loss.item())
#         # ===================backward====================
#         optimizer2.zero_grad()
#         loss.backward()
#         optimizer2.step()
#         #====================new=========================
# #         model.eval()
# #         z = model.encoder(img).detach_()
# #         output = output.detach_()
# #         output_a = aux(z)
# #         loss_a = (criterion(output_a, output).mul(temperature).exp())*criterion(output_a, img)
# #         tot_loss_a.append(loss_a)
# #         optimizer_a.zero_grad()
# #         loss_a.backward()
# # #         optimizer_a.step()
#     scheduler.step()
#     # ===================save_best====================
#     mean_loss = np.mean(tot_loss)
#     if mean_loss < best_loss:
#         best_loss = mean_loss
#         torch.save(model2, 'best_model_{}.pt'.format(model_type2))
# #         torch.save(aux, 'best_model_{}.pt'.format('aux'))
#     # ===================log========================
#     qqdm_train.set_infos({
#         'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
#         'loss': f'{mean_loss:.4f}',
#     })
#     # ===================save_last========================
#     torch.save(model2, 'last_model_{}.pt'.format(model_type2))

In [ ]:
# best_loss = np.inf
# model3.train()
# # aux.train()
# num_epochs = 80
# qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
# scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer3, T_0 = 2, T_mult = 2)
# for epoch in qqdm_train:
#     tot_loss = list()
#     tot_loss_a = list()
#     for data in train_dataloader:
#         temperature = epoch // 2 + 1
#         # ===================loading=====================
#         img = data.float().cuda()
#         if model_type3 in ['fcn']:
#             img = img.view(img.shape[0], -1)
        

#         # ===================forward=====================
#         output = model3(img)
#         if model_type3 in ['vae']:
#             loss = loss_vae(output[0], img, output[1], output[2], criterion)
#         else:
#             loss = criterion(output, img)

#         tot_loss.append(loss.item())
#         # ===================backward====================
#         optimizer3.zero_grad()
#         loss.backward()
#         optimizer3.step()
#         #====================new=========================
# #         model.eval()
# #         z = model.encoder(img).detach_()
# #         output = output.detach_()
# #         output_a = aux(z)
# #         loss_a = (criterion(output_a, output).mul(temperature).exp())*criterion(output_a, img)
# #         tot_loss_a.append(loss_a)
# #         optimizer_a.zero_grad()
# #         loss_a.backward()
# # #         optimizer_a.step()
#     scheduler.step()
#     # ===================save_best====================
#     mean_loss = np.mean(tot_loss)
#     if mean_loss < best_loss:
#         best_loss = mean_loss
#         torch.save(model3, 'best_model_{}.pt'.format(model_type3))
# #         torch.save(aux, 'best_model_{}.pt'.format('aux'))
#     # ===================log========================
#     qqdm_train.set_infos({
#         'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
#         'loss': f'{mean_loss:.4f}',
#     })
#     # ===================save_last========================
#     torch.save(model3, 'last_model_{}.pt'.format(model_type3))

# Classifier

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [31]:
eval_batch_size = 1024

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
checkpoint_path2 = f'last_model_{model_type2}.pt'
checkpoint_path3 = f'last_model_{model_type3}.pt'
# checkpoint_path = '/kaggle/input/hw8-pt/ml2023spring-hw8/best_model_resnet.pt'
checkpoint_path_a = 'last_model_aux.pt'
# checkpoint_path = '/kaggle/input/hw-model/best_model_cnn-2pt'
model = torch.load(checkpoint_path)
model.eval()
# aux = Auxiliary().to('cuda')
# aux = torch.load(checkpoint_path_a)
# model2 = torch.load(checkpoint_path2)
# model2.eval()
# model3 = torch.load(checkpoint_path3)
# model3.eval()
# aux = torch.load(checkpoint_path_a)
# aux.eval()
# prediction file 
out_file = '/kaggle/working/prediction.csv'
out_file_a = '/kaggle/working/prediction_a.csv'

In [33]:
anomality = []
auxs = []
with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        img = data.float().to('cuda')
        z = model.encoder(img)
        output = model(img)   
#         output, output_a = model.decoder(z), aux(z)
        loss = eval_loss(output, img[:, 0:24, :, :]).mean([1, 2, 3])
#         loss_a = eval_loss(output_a, img).mean([1, 2, 3])
        anomality.append(loss)
#         auxs.append(loss_a)
        
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()
# auxs = torch.cat(auxs, axis=0)
# auxs = torch.sqrt(auxs).reshape(len(test), 1).cpu().numpy()
 
df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')
# df_a = pd.DataFrame(auxs, columns=['score'])
# df_a.to_csv(out_file_a, index_label = 'ID')
# anomality_ = []
# for i in range(0, len(anomality)):
#     anomality_.append((anomality[i]+auxs[i])/2)
# df = pd.DataFrame(anomality_, columns=['score'])
# df.to_csv('/kaggle/working/prediction_sum.csv', index_label = 'ID')
# df = pd.DataFrame(auxs, columns=['score'])
# df.to_csv(out_file_a, index_label = 'ID')
    


In [ ]:
# anomality_ = []
# for i in range(0, len(anomality)):
#     anomality_.append((anomality[i]+anomality2[i]+anomality3[i])/3)
# df = pd.DataFrame(anomality_, columns=['score'])
# df.to_csv('/kaggle/working/prediction_sum.csv', index_label = 'ID')
# # df = pd.DataFrame(auxs, columns=['score'])
# # df.to_csv(out_file_a, index_label = 'ID')